# DPD simulation of polymer solution

**`By Yuanhui Tang`**

## Introduction

This tutorial is designed to provide an introduction to DPD simulations with GALAMOST. It is assumed that you have already got a basic knowledge of the Linux command line (if not, you can begin with this little learning program: BEGINNER'S CORNER: Learning the [Unix Command-line](https://www.codecademy.com/learn/learn-the-command-line)) and DPD principle before. It is suitable for new users who want to learn about how to run DPD simulations. It does however assume that you have a machine with GALAMOST correctly installed.

In this tutorial, we would like to make a system consisting of polymers and solvents, and we want to simulate the phase separation in the system as the temperature is quenching or decreasing.

**Download [running files](https://bitbucket.org/galamostdevelopergroup/source-code/src/master/examples/Case3-DPD-Polymer-Solution/)**.

In order to run this DPD simulation with GALAMOST, two files should be prepared:
*  `xml`: the file that describes all the information (including position, velocity, topology, etc.) of the particles in the system, more about [XML data format](https://galamost.readthedocs.io/en/latest/data-format.html); the xml configuration file could be generated by `Molgen` plugin
*  `gala`: The file that describes the settings for the GALAMOST simulation engine

## Prepare topology and coordinate files

For this tutorial, you will build a system containing two kinds of molecules (shown as below) for simulation.
In order to build the two molecules, you need to write a python script of [`Molgen`](https://galamost.readthedocs.io/en/latest/molgen.html) that will run on a terminal of Linux to call the [`Molgen`](https://galamost.readthedocs.io/en/latest/molgen.html) plugin of GALAMOST.
The script "polymer.molg" to build a polymer-solvent system is shown as below in blue

In [ ]:
#!/usr/bin/python  # python language script
import sys      
sys.path.append('/opt/galamost3/lib')  #set the path which the lib is
import molgen      # import the module molgen for build the system

mol1=molgen.Molecule(40)     #build a polymer molecule consisting of 40 repeat units
mol1.setParticleTypes("A*40")  #the polymer type is "A"

topo="0-1"                 
for i in range(1,39):             
	topo +=","+str(i)+"-"+str(i+1) #set the topology of the polymer: the polymer particles are connected linearly

mol1.setTopology(topo)
mol1.setBondLength(4.0)    #adopt the basic harmonic bond potential for the bond stretching 
mol1.setMass(1.0)          #set the polymer mass to be 1 (normal setting)

mol2=molgen.Molecule(1)#number
mol2.setParticleTypes("C")
mol2.setMass(1.0)

gen=molgen.Generators(100,100,100) #set the box size is 100x100x100
gen.addMolecule(mol1,15000)         #put 15000 polymer molecules 
gen.addMolecule(mol2, 2400000)   #put 2400000 solvent particles with chain length is one
gen.outPutXml("config0_size100_N40_20")       # output the xml file

## Prepare GALAMOST DPD gala input files

Next component needed is the input files that define the program settings for each DPD run. For this system, firstly, we will perform an equilibrium run on the system at a high temperature of $k_BT$ = 1.73, then decrease the system temperature to induce phase separation.
The gala file "polymer.gala" is shown as below in blue:

In [ ]:
# the default box is with periodic boundary conditions
#!/usr/bin/python 
import sys
sys.path.append('/opt/galamost3/lib') 
import galamost                      # import the module galamost 
from optparse import OptionParser   
 
global _options                        
parser = OptionParser()                 
parser.add_option('--gpu', dest='gpu',help='GPU on which to execute')  #set the GPU
(_options, args) = parser.parse_args()      
 

filename = 'config0_size100_N40_20.xml'   # input the xml file obtained by the molgen module
randomnum = 12340                   # random number seed
build_method = galamost.XmlReader(filename)  # read the initial file xml
perform_config = galamost.PerformConfig(_options.gpu)
all_info = galamost.AllInfo(build_method,perform_config)
 
dt = 0.01     # set the time step, can be increased to 0.04, a larger time step results in a not good temperature control
app = galamost.Application(all_info, dt)  

neighbor_list = galamost.NeighborList(all_info, 1.0 ,0.05)          
dpd = galamost.DpdForce(all_info, neighbor_list, 1.0, randomnum)   
app.add(dpd)                         # add the dpd module
 
v = galamost.VariantLinear()           # set a linear algorithm for control the temperature 
v.setPoint(1000,1.73)       # set the temperature = 1.73 when the step = 1000
v.setPoint(10000,1.73)     # set the temperature keeps to be 1.73 when the step = 10000
v.setPoint(100000,1.00)    # set the temperature decreases to be 1.00 when the step = 100000
dpd.setT(v)

bondforce = galamost.BondForceHarmonic(all_info)  # set the harmonic bond stretching
bondforce.setParams('A-A', 4.0, 0.0)        # 4: spring constant; 0:  equilibrium length
app.add(bondforce)                   

groupAC = galamost.ParticleSet(all_info, ["A","C"])  #build a group that includes particle "A" and "C"
comp_infoAC = galamost.ComputeInfo(all_info, groupAC)  #make a information group for the #group "AC"

Gwvv = galamost.DpdGwvv(all_info, groupAC)       #set the GWVV algorithm to the particle AC
app.add(Gwvv)

sort_method = galamost.Sort(all_info)             #optimizing the video memory
sort_method.setPeriod(80)
app.add(sort_method)


DInfo = galamost.DumpInfo(all_info, comp_infoAC, 'dataAC.log')  #export the information to the file "dataAC.log"
DInfo.setPeriod(2000)  #export the information of the system every 2000 steps
app.add(DInfo)        
 
dcd = galamost.DcdDump(all_info, 'particles',True)   # export the configuration information for making a movie
dcd.setPeriod(5000)    # export the information of the system every 5000 steps
dcd.unpbc(True)   # remove PBC 
app.add(dcd)
 
xml = galamost.XmlDump(all_info, 'particles')    # output the configuration xml file
xml.setPeriod(5000)  # export the information of the system every 5000 steps
xml.setOutputBond(True)       #export bond information
app.add(xml)

#ready ro run
# Period1: start the highT temperature to equilibrate the system
dpd.setParams('A', 'A', 43.25, 3.0)   # #set the alpha interaction parameters
dpd.setParams('A', 'C', 26.19, 3.0)   # #between species
dpd.setParams('C', 'C', 43.25, 3.0)   #(type,type,alpha,sigma)
app.run(100000)                # run 100000 time steps

# Period2: decrease the temperature

dpd.setParams('A', 'A', 43.3, 3.0)   
dpd.setParams('A', 'C', 45.3, 3.0)   
dpd.setParams('C', 'C', 43.3, 3.0)   
app.run(90000)
neighbor_list.printStats() 

[OVITO](http://www.ovito.org/) is suggested to render the pictures of system snapshots with the output XML files